In [3]:
!pip install -r requirements.txt

  Attempting uninstall: strands-agents
    Found existing installation: strands-agents 1.10.0
    Uninstalling strands-agents-1.10.0:
      Successfully uninstalled strands-agents-1.10.0


# Autonomous AI Advertising Agent with Crypto Payments

## ⚠️ **SAFETY WARNINGS - READ BEFORE PROCEEDING**

### 🚨 **TESTNET ONLY - DO NOT USE REAL FUNDS**
- **This notebook uses BASE SEPOLIA TESTNET** - never use mainnet
- **Use only TEST TOKENS** - do not fund with real cryptocurrency
- **Wallet credentials are for development only** - never use production keys
- **Educational demonstration** - not intended for production use

### 🔒 **Security Best Practices**
- ✅ Use separate test credentials and API keys
- ✅ Verify you're on Base Sepolia testnet (not mainnet)
- ✅ Only use testnet USDC tokens (no real value)
- ❌ Never expose real wallet private keys
- ❌ Never run this code with production credentials
- ❌ Never use mainnet addresses or real cryptocurrency

---
## Overview

This notebook demonstrates how to build an **autonomous AI advertising agent** that can create complete marketing campaigns while automatically paying for premium services using cryptocurrency. By combining cutting-edge AI agent frameworks with blockchain technology, we create a system that operates independently in the digital economy.

## What This Notebook Demonstrates

### 🤖 **Autonomous Agent Architecture**
- Built using the **Strands AI framework** with Claude Sonnet 4
- Integrated with **Coinbase AgentKit** for blockchain functionality
- Capable of making independent decisions and executing complex workflows

### 💳 **Crypto-Native Payments**
- **X402 Payment Protocol**: Automatic micropayments for API services
- **Base Sepolia Network**: EVM-compatible blockchain transactions
- **USDC Payments**: Stable currency for predictable service costs
- **Autonomous Wallet Management**: No human intervention required

### 🎨 **Premium Service Integration**
- **AI Image Generation**: Creates custom visuals for ad campaigns
- **Weather Data APIs**: Enables location and season-based targeting
- **Payment-Gated Access**: Demonstrates real economic interactions

### 📊 **Complete Campaign Creation**
- **Multi-Platform Ads**: Social media, display, email, and print formats
- **Dynamic Content**: Adapts messaging based on real-time data
- **Visual Integration**: Embeds generated images into campaign materials
- **HTML Output**: Produces ready-to-deploy advertising assets

## Key Innovation: Economic Agency

This agent represents a new paradigm where AI systems can:
- 🏦 **Manage their own budgets** for operational expenses
- 🛒 **Purchase services** needed to complete tasks
- 🔄 **Optimize spending** based on campaign requirements
- 📈 **Scale operations** without human financial oversight

## Real-World Applications

- **Marketing Agencies**: Automated campaign generation and optimization
- **E-commerce**: Dynamic product advertising with seasonal adaptations
- **Local Businesses**: Location-based campaigns with weather integration
- **Content Creators**: Automated visual and copy generation workflows

## Technical Architecture

The notebook showcases integration between:
- **Strands AI Framework** → Agent reasoning and tool orchestration
- **Coinbase AgentKit** → Blockchain wallet and transaction management
- **X402 Protocol** → HTTP-based micropayment standard
- **External APIs** → Premium image generation and weather services

---

**Run through this notebook to see how autonomous agents can operate as independent economic actors in the digital advertising ecosystem!**

In [1]:
import json
import sys
import time
import os
import botocore.exceptions
from coinbase_agentkit import (
    AgentKit,
    AgentKitConfig,
    CdpEvmWalletProvider,
    CdpEvmWalletProviderConfig,
    cdp_api_action_provider,
    wallet_action_provider,
    x402_action_provider
)

from typing import Dict, Any, Optional, List
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import image_reader
import base64
from pathlib import Path
from coinbase_agentkit_strands_agents import get_strands_tools
from strands.tools import normalize_tool_spec

In [2]:
# Replace with your CDP test wallet. Only use an empty test wallet.
os.environ["CDP_API_KEY_ID"] = "CDP API Key"
os.environ["CDP_API_KEY_SECRET"] = "CDP API SECRET"
os.environ["CDP_WALLET_SECRET"] = "CDP WALLET SCRET"

# Wallet Provider and AgentKit Setup

This cell initializes the core blockchain functionality for our advertising agent:

## Wallet Provider Configuration
- **CdpEvmWalletProvider**: Creates an EVM-compatible wallet for blockchain transactions
- **Credentials**: Loaded securely from environment variables (CDP API keys and wallet secret)
- **Network**: Defaults to Base Sepolia testnet for development

## AgentKit Initialization
The AgentKit combines multiple action providers:
- **CDP API Provider**: Direct blockchain API interactions
- **Wallet Provider**: Transaction signing and wallet operations  
- **X402 Provider**: HTTP payment protocol for accessing paid APIs

This setup enables our agent to:
- 💰 Make cryptocurrency payments for premium services
- 🔗 Interact with blockchain networks
- 🌐 Access payment-gated APIs using the X402 protocol
- 📝 Manage wallet transactions autonomously

In [3]:
# Initialize the wallet provider with the config
wallet_provider = CdpEvmWalletProvider(
        CdpEvmWalletProviderConfig(
            api_key_id=os.getenv("CDP_API_KEY_ID"),  # CDP API Key ID
            api_key_secret=os.getenv("CDP_API_KEY_SECRET"),  # CDP API Key Secret
            wallet_secret=os.getenv("CDP_WALLET_SECRET"),  # CDP Wallet Secret
            # network_id=config.network_id,  # Network ID - Optional, will default to 'base-sepolia'
            # address=config.address,  # Wallet Address - Optional, will trigger idempotency flow if not provided
            # idempotency_key=config.idempotency_key,  # Idempotency Key - Optional, seeds generation of a new wallet
        )
    )


# Create AgentKit instance with wallet and action providers
agentkit = AgentKit(
        AgentKitConfig(
            wallet_provider=wallet_provider,
            action_providers=[
                cdp_api_action_provider(),
                wallet_action_provider(),
                x402_action_provider()
            ],
        )
    )

# Service Discovery System and Custom Tools

This cell creates a service catalog and custom tools for external API integration:

## Service Catalog
Defines available external services, that matches the paid service endpoints, with complete schemas:

### 🎨 Media Creation Service
- **Purpose**: AI image generation for ad visuals
- **Cost**: Payment required via X402 protocol
- **Input**: Text prompts describing desired images
- **Output**: Generated image URLs/paths

### 🌤️ Weather Service  
- **Purpose**: Real-time weather data for location-based campaigns
- **Cost**: Payment required via X402 protocol
- **Input**: City name and unit preferences
- **Output**: Current weather conditions and forecasts

## Custom Agent Tools

### `@tool list_available_services()`
- Lists all available external services
- Helps agent discover capabilities dynamically

### `@tool get_service_schema(service_name)`
- Returns detailed API schema for specific services
- Enables proper request formatting

### `@tool create_ad_html(ad_html_content, file_name, output_path)`
- Generates HTML files containing complete ad campaigns
- Embeds images and formats content for web display
- Creates organized file structure for campaign assets

These tools enable intelligent service discovery and seamless integration with payment-gated APIs.

In [4]:
# Configuration
BASE_URL = os.getenv("BASE_URL", "http://0.0.0.0:4021") # Replace port number with actual service port if modified

# Service Discovery Functions
def external_service_catalog(service_name: str) -> Optional[Dict[str, Any]]:
    """
    Returns service endpoint information including schemas and URLs.

    Args:
        service_name: Name of the service to query

    Returns:
        Dictionary containing service metadata, schemas, and endpoint URL
    """
    catalog = {
        "media-creation": {
            "name": "Image Generator",
            "description": "Generate images based on text prompts using AI. Perfect for creating ad visuals, product shots, and marketing materials.",
            "url": f"{BASE_URL}/image_generator",
            "method": "POST",
            "payment_required": True,
            "input_schema": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The prompt describing what images to generate",
                        "example": "Generate 3 images of a friendly dragon reading a book",
                        "minLength": 1,
                        "maxLength": 500
                    }
                },
                "required": ["query"]
            }
        },

        "weather": {
            "name": "Weather Information",
            "description": "Get current weather information for any city. Useful for location-based or seasonal ad campaigns.",
            "url": f"{BASE_URL}/weather",
            "method": "POST",
            "payment_required": True,
            "input_schema": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City name (e.g., 'London' or 'London,UK')",
                        "example": "London,UK"
                    },
                    "units": {
                        "type": "string",
                        "description": "Units: 'metric', 'imperial', or 'standard'",
                        "default": "metric"
                    }
                },
                "required": ["city"]
            }
        }
    }

    return catalog.get(service_name)


# Custom Tools for the Agent
@tool
def list_available_services() -> str:
    """
    List all available external services that can be used for ad creation.
    Returns information about available endpoints including weather data and image generation.
    Use this when you need to know what external services are available for your ad campaign.
    """
    services = {
        "media-creation": external_service_catalog("media-creation"),
        "weather": external_service_catalog("weather")
    }

    result = "Available Services for Content Creation:\n\n"

    for service_name, service_info in services.items():
        if service_info:
            result += f"Service: {service_info['name']}\n"
            result += f"  Description: {service_info['description']}\n"
    return result


@tool
def get_service_schema(service_name: str) -> str:
    """
    Get detailed schema information and URL endpoint for a specific service.

    Args:
        service_name: Name of the service ('media-creation' or 'weather')

    Returns:
        Detailed schema information including input requirements
    """
    service = external_service_catalog(service_name)

    if not service:
        return f"Service '{service_name}' not found. Available services: 'media-creation', 'weather'"

    result = f"Service: {service['name']}\n"
    result += f"Description: {service['description']}\n\n"
    result += f"Endpoint: {service['method']} {service['url']}\n\n"
    result += f"Input Schema:\n{json.dumps(service['input_schema'], indent=2)}\n"
    
    return result


@tool
def create_ad_html(
    ad_html_content: str,
    file_name: str,
    output_path: str = "ad_campaign",
) -> str:
    """
    Create an HTML file displaying the ad campaign.

    Args:
        ad_html_content: Html data containing ad content including generated images (if used) with html syntax used in creating the html file
        file_name: name of the html file including the .html extension
        output_path: Path where HTML file will be saved

    Returns:
        Path to the created HTML file
    """
    # Write to file
    output_file = Path(f"{output_path}/{file_name}")
    output_file.parent.mkdir(parents=True, exist_ok=True)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(ad_html_content)

    return f"HTML file saved to {str(output_file.absolute())}"


# Tool Schema Modification for X402 Payment Protocol

This cell modifies tool schemas to ensure compatibility with HTTP payment protocols:

## Schema Modification Function
The `modify_tool_schema()` function updates tool parameter types:
- **Target Field**: Changes 'headers' parameter from 'string' to 'object' type
- **Normalization**: Ensures schemas comply with Strands tool specifications

## Tools Modified
- `x402ActionProvider_retry_with_x402`
- `x402ActionProvider_make_http_request` 
- `x402ActionProvider_make_http_request_with_x402`

## Why This Matters
X402 payment protocol requires properly formatted HTTP headers as objects rather than strings. This modification ensures our agent can:
- ✅ Successfully make payment-gated API requests
- ✅ Pass authentication headers correctly
- ✅ Handle retry logic for payment failures
- ✅ Integrate seamlessly with paid external services

Without this modification, payment requests would fail due to schema mismatches.

In [5]:
def modify_tool_schema(tool_spec, field_name, new_field_type, new_description=None):
    """Modify a specific field type in a tool's schema"""
    modified_spec = tool_spec.copy()
    
    # Navigate to the field in inputSchema
    if "inputSchema" in modified_spec and "json" in modified_spec["inputSchema"]:
        schema = modified_spec["inputSchema"]["json"]
        if "properties" in schema and field_name in schema["properties"]:
            # Modify the field type
            schema["properties"][field_name]["type"] = new_field_type
            
            # Optionally modify description
            if new_description:
                schema["properties"][field_name]["description"] = new_description
    
    # Normalize the modified schema
    return normalize_tool_spec(modified_spec)

# Load AgentKit actions as Strands tools
tool_list = get_strands_tools(agentkit)

# List of tools to modify their schema to match the expected action schema
tool_to_modify = ["x402ActionProvider_retry_with_x402", "x402ActionProvider_make_http_request", "x402ActionProvider_make_http_request_with_x402"]

modified_tools = []
for tool in tool_list:
    tool_spec = tool.tool_spec
    
    # Modify specific field - change 'config' from 'string' to 'object'
    if tool.tool_name in tool_to_modify:
        print(f"Modifying schema of {tool.tool_name}")
        modified_spec = modify_tool_schema(
            tool_spec, 
            field_name="headers",
            new_field_type="object",
        )
        # Update the tool's spec
        tool._tool_spec = modified_spec
    
    modified_tools.append(tool)

Modifying schema of x402ActionProvider_make_http_request
Modifying schema of x402ActionProvider_make_http_request_with_x402
Modifying schema of x402ActionProvider_retry_with_x402


# Advertising Agent Configuration

This cell sets up our specialized advertising agent with comprehensive capabilities:

## Model Configuration (Optional)
The commented BedrockModel shows advanced configuration options:
- **Claude Sonnet 4**: Latest high-performance reasoning model
- **Thinking Budget**: 2048 tokens for internal reasoning
- **Cache Settings**: Optimized for prompt and tool caching

## Tool Integration
Combines multiple tool categories:
- **Custom Tools**: Service discovery and HTML generation
- **Strands Tools**: Image reading and processing
- **AgentKit Tools**: Blockchain and payment capabilities
- **Modified X402 Tools**: Payment-gated API access

## System Prompt Design
The comprehensive system prompt defines the agent as an expert in:

### 🎯 Core Capabilities
- Digital marketing and copywriting
- Multi-platform ad creation (social, display, email, print)
- Visual content generation and integration
- Campaign strategy and optimization

### 🛠️ Technical Workflow
1. **Discovery**: Use service catalog to find relevant APIs
2. **Enhancement**: Generate visuals and gather contextual data
3. **Creation**: Craft compelling copy with clear CTAs
4. **Integration**: Combine all elements into cohesive campaigns
5. **Output**: Generate complete HTML files with embedded media

### ✅ Quality Standards
- Mobile-first design approach
- A/B testing recommendations
- Proper image embedding in HTML
- Brand consistency guidelines
- Performance optimization

This agent is designed to create production-ready advertising campaigns autonomously.

In [6]:
# Initialize the Bedrock Model

# bedrock_model = BedrockModel(
#     cache_prompt="default",
#     cache_tools="default",
#     model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
#     max_tokens=4000,
#     additional_request_fields={
#         "thinking": {
#             "type": "enabled",
#             "budget_tokens": 2048 # Minimum of 1,024
#         }
#     }
# )


# load coinbase agentkit actions as strands agent tools
tool_list = get_strands_tools(agentkit)

# Create the Ads Creative Agent
advertising_agent = Agent(
    # model=bedrock_model,  # defaults to Sonnet 4
    tools=[
        list_available_services,
        get_service_schema,
        image_reader,
        create_ad_html,
    ] + modified_tools, # Custom tools plus Coinbase Agentkit tools
    system_prompt="""You are an expert advertising and marketing content creator with extensive experience in digital marketing, copywriting, and creative direction.

Your capabilities:
1. Create compelling ad copy for various platforms (social media, display, email, print)
2. Discover and use external services for ad enhancement:
   - Generate professional visuals using the image generation service
   - Get real-time weather data for location-based and seasonal campaigns
3. Design complete ad campaigns with copy, visuals, and targeting strategy
4. Interacting with on chain actions for making and necessary payments
5. Tool for creating a html file of the generated ads

When creating ads:
- First start with and outline of the ads to be created
- Use list_available_services() to discover external services that can help in creating ads
- Consider the target audience, platform, and campaign goals
- If location-specific or weather-dependent, use the appropiate external service to get weather info
- Always generate relevant visuals using image endpoint service
- Use persuasive copywriting techniques (AIDA, PAS, FAB)
- Include clear call-to-action (CTA)
- Optimize for the specific ad format and platform
- Save the ads content as a html file. When creating the html content, make sure images are rendered properly. Take this part very seriously

Ad Formats You Can Create:
- Social Media Ads (Facebook, Instagram, LinkedIn, Twitter)
- Display Banner Ads
- Email Marketing
- Print Advertisements
- Video Ad Scripts
- Search Ads (Google, Bing)

Response Structure:
1. Campaign Overview (objective, audience, platform)
2. Headline(s) - attention-grabbing and benefit-focused
3. Body Copy - persuasive and engaging
4. Call-to-Action - clear and action-oriented
5. Visual Description/Generated Images
6. Targeting Recommendations (demographics, interests, behaviors)
7. Budget & Timing Suggestions (if applicable)
8. **HTML File Creation Checklist:**
   - [ ] All images generated and URLs obtained
   - [ ] Images embedded in HTML with <img> tags using relative paths
   - [ ] HTML file saved with rendered images

Best Practices:
- Keep it concise and scannable
- Focus on benefits, not just features
- Use emotional triggers and power words
- Create urgency when appropriate
- Ensure brand consistency
- Mobile-first approach
- A/B testing recommendations
- Include images in teh generated html file

CRITICAL REQUIREMENT - HTML FILE GENERATION:
When saving ads as HTML files, you MUST:
✓ Include ALL generated images embedded in the HTML
✓ Use proper <img> tags with src attributes pointing to the image URLs
✓ Ensure images are visible when the HTML file is opened
✓ Test that image URLs are accessible and properly formatted
✓ Never create HTML files with missing or broken image references
**NEVER use absolute file system paths:**
❌ `<img src="/home/user/file-path/output/image.png"...>`

**ALWAYS use relative paths based on the HTML file location:**
✅ `<img src="../output/image.png"...>`


BEFORE saving the HTML file, verify:
1. Each ad section has its corresponding image embedded
2. Image URLs are complete and valid
3. Images have appropriate alt text
4. The HTML renders images correctly

Be creative, data-driven, and results-oriented!"""
)

In [7]:
response = advertising_agent(""" Generate an ad for the following:
    product="ice cream shop promotion",
    city="Miami,US",
    platform="social-media"
""")

I'll create a compelling social media ad for your ice cream shop promotion in Miami! Let me start by outlining the campaign and then gathering the resources I need.

## Campaign Overview
- **Objective**: Drive foot traffic to ice cream shop in Miami
- **Audience**: Local Miami residents, tourists, families, young adults
- **Platform**: Social Media (Facebook, Instagram)
- **Strategy**: Weather-responsive promotion leveraging Miami's warm climate

Let me first check what external services are available to enhance this ad:
Tool #1: list_available_services
Perfect! Let me get the current weather in Miami to create a weather-responsive ad, and then I'll generate compelling visuals.
Tool #2: get_service_schema

Tool #3: x402ActionProvider_make_http_request
I need to make a payment to access the weather service. Let me check my wallet balance and proceed with the payment:
Tool #4: WalletActionProvider_get_wallet_details
Let me request some test funds to cover the payment:
Tool #5: CdpApiActi

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '# 🍦 MIAMI ICE CREAM SHOP AD CAMPAIGN - COMPLETE! \n\n## ✅ **HTML File Creation Checklist - VERIFIED:**\n- [x] All 5 generated images embedded in HTML with proper paths\n- [x] Images use relative paths (`../output/image.png`)\n- [x] HTML file saved with rendered images\n- [x] Weather-responsive content integrated\n- [x] Multiple CTA options included\n- [x] Mobile-responsive design implemented\n\n## 🎯 **Campaign Summary**\n\n**Perfect Timing Alert!** 🌡️ Miami is currently at an ideal **80°F with few clouds** - perfect ice cream weather conditions!\n\n### **Headline Options (A/B Test Ready)**\n1. **Primary**: "80°F in Miami = Perfect Ice Cream Weather! 🌴"\n2. **Alternative**: "Beat the Miami Heat with Our Tropical Flavors!"\n3. **Urgency**: "Cool Down Miami Style - Tropical Scoops Await!"\n\n### **Key Messaging Strategy**\n- **Weather-Responsive**: Leverages real-time 80°F temperature\n- **Local Appeal